In [1]:
! ls covid19-opendata-vaccini/dati/*.csv

covid19-opendata-vaccini/dati/anagrafica-vaccini-summary-latest.csv
covid19-opendata-vaccini/dati/consegne-vaccini-latest.csv
covid19-opendata-vaccini/dati/punti-somministrazione-latest.csv
covid19-opendata-vaccini/dati/punti-somministrazione-tipologia.csv
covid19-opendata-vaccini/dati/somministrazioni-vaccini-latest.csv
covid19-opendata-vaccini/dati/somministrazioni-vaccini-summary-latest.csv
covid19-opendata-vaccini/dati/vaccini-summary-latest.csv


In [9]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

if not os.path.isdir('DW'): os.mkdir('DW')

# mapping columns names
def createNameMappingDict(df):
    '''This function returns a dictionary which helps mapping columns names in a DataFrame'''
    nameMappingDict = {oldName : oldName.replace('_',' ').title() for oldName in df.columns}
    
    return nameMappingDict

def ETL_anagraficaVacciniSummaryLatest():
    global anaVacSumLat 
    anaVacSumLat = pd.read_csv('covid19-opendata-vaccini/dati/anagrafica-vaccini-summary-latest.csv')
    anaVacSumLat = anaVacSumLat.rename(columns=createNameMappingDict(anaVacSumLat))
    
    vLastUpdate = datetime.strptime(anaVacSumLat.iloc[0,-1],"%Y-%m-%d").strftime("%d/%m/%Y")
    
    anaVacSumLat = anaVacSumLat.iloc[:,:-1]
    
    # -----NEW COLUMNS-----
    anaVacSumLat['% Seconda Dose Sul Totale'] = round(100 * anaVacSumLat['Seconda Dose']/anaVacSumLat['Totale'], 2)
    anaVacSumLat['Platea'] = [2298846,6084382,6854632,8937229,9414195,7364364,5968373,3628160,613523]
    anaVacSumLat['% Seconda Dose Assoluta'] = round(anaVacSumLat['Seconda Dose']/anaVacSumLat['Platea'] * 100,2)
    anaVacSumLat['% Totale Assoluto'] = round(anaVacSumLat['Totale']/anaVacSumLat['Platea'] * 100,2)
    # ---------------------
    
    anaVacSumLat.to_csv('DW/anagraficaVacciniSummaryLatest.csv')
    

def ETL_consegneVacciniLatest():
    global consVacciniLat 
    consVacciniLat = pd.read_csv('covid19-opendata-vaccini/dati/consegne-vaccini-latest.csv')
    consVacciniLat = consVacciniLat.rename(columns=createNameMappingDict(consVacciniLat)
                                          ).rename(columns={'Nome Area': 'Regione'} )   
    consVacciniLat = consVacciniLat.iloc[:,[1,2,3,7]]
    consVacciniLat['Data Consegna'] = pd.to_datetime(consVacciniLat['Data Consegna'])
    
    consVacciniLat.to_csv('DW/consegneVacciniLatest.csv')

def ETL_somministrazioniVacciniSummaryLatest():
    global somVacciniSumLat
    somVacciniSumLat = pd.read_csv('covid19-opendata-vaccini/dati/somministrazioni-vaccini-summary-latest.csv')
    somVacciniSumLat = somVacciniSumLat.rename(columns=createNameMappingDict(somVacciniSumLat)
                                              ).rename(columns={'Nome Area': 'Regione'} )

    somVacciniSumLat.drop(columns=['Area', 'Codice Nuts1', 'Codice Nuts2', 'Codice Regione Istat'], inplace=True)

    somVacciniSumLat['Data Somministrazione'] = pd.to_datetime(somVacciniSumLat['Data Somministrazione']).dt.date
    
    # -----NEW COLUMNS-----
    somVacciniSumLat['Totale'] = somVacciniSumLat['Prima Dose'] + somVacciniSumLat['Seconda Dose']
    # ---------------------
    
    somVacciniSumLat = somVacciniSumLat.sort_values(['Data Somministrazione','Regione']).reset_index()
    somVacciniSumLat = somVacciniSumLat.drop(columns='index')
    somVacciniSumLat.to_csv('DW/somministrazioniVacciniSummaryLatest.csv')

ETL_anagraficaVacciniSummaryLatest()
ETL_consegneVacciniLatest()
ETL_somministrazioniVacciniSummaryLatest()

In [8]:
somVacciniSumLat

,Data Somministrazione,Totale,Sesso Maschile,Sesso Femminile,Categoria Operatori Sanitari Sociosanitari,Categoria Personale Non Sanitario,Categoria Altro,Categoria Ospiti Rsa,Categoria Over80,Categoria Forze Armate,Categoria Personale Scolastico,Prima Dose,Seconda Dose,Regione
0,2020-12-27,37,19,18,34,0,1,2,0,0,0,37,0,Abruzzo
1,2020-12-27,106,41,65,104,0,2,0,0,0,0,106,0,Basilicata
2,2020-12-27,275,150,125,274,0,1,0,0,0,0,275,0,Calabria
3,2020-12-27,720,422,298,703,14,3,0,0,0,0,720,0,Campania
4,2020-12-27,968,372,596,824,1,117,26,0,0,0,968,0,Emilia-Romagna
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773,2021-03-24,7572,3624,3948,350,5,3979,232,2618,35,353,4920,2652,Sicilia
1774,2021-03-24,4599,1869,2730,51,157,818,39,3401,34,99,2099,2500,Toscana
1775,2021-03-24,1824,889,935,108,76,90,4,1493,31,22,1021,803,Umbria
1776,2021-03-24,212,96,116,2,0,38,0,169,2,1,11,201,Valle d'Aosta / Vallée d'Aoste


In [10]:
anaVacSumLat

,Fascia Anagrafica,Totale,Sesso Maschile,Sesso Femminile,Categoria Operatori Sanitari Sociosanitari,Categoria Personale Non Sanitario,Categoria Altro,Categoria Ospiti Rsa,Categoria Over80,Categoria Forze Armate,Categoria Personale Scolastico,Prima Dose,Seconda Dose,% Seconda Dose Sul Totale,Platea,% Seconda Dose Assoluta,% Totale Assoluto
0,16-19,8878,4250,4628,4006,658,2896,1159,0,54,105,6039,2839,31.98,2298846,0.12,0.39
1,20-29,591954,210451,381503,405492,40346,65085,9600,0,17821,53610,368260,223694,37.79,6084382,3.68,9.73
2,30-39,857977,335737,522240,520665,67754,104846,12265,0,33415,119032,557215,300762,35.05,6854632,4.39,12.52
3,40-49,1152749,403571,749178,602555,106262,153670,18390,0,61895,209977,783284,369465,32.05,8937229,4.13,12.90
4,50-59,1486370,528941,957429,747095,152362,198238,30500,0,83247,274928,1004397,481973,32.43,9414195,5.12,15.79
5,60-69,910437,430285,480152,500964,75216,145997,37416,0,11553,139291,598904,311533,34.22,7364364,4.23,12.36
6,70-79,387671,220877,166794,99295,15134,202561,68433,0,99,2149,294041,93630,24.15,5968373,1.57,6.50
7,80-89,2332287,997986,1334301,13948,6444,64246,186667,2060443,326,213,1658878,673409,28.87,3628160,18.56,64.28
8,90+,618122,188611,429511,2758,2151,12836,158467,441805,71,34,417840,200282,32.40,613523,32.64,100.75


In [11]:
somVacciniSumLat

,Data Somministrazione,Totale,Sesso Maschile,Sesso Femminile,Categoria Operatori Sanitari Sociosanitari,Categoria Personale Non Sanitario,Categoria Altro,Categoria Ospiti Rsa,Categoria Over80,Categoria Forze Armate,Categoria Personale Scolastico,Prima Dose,Seconda Dose,Regione
0,2020-12-27,37,19,18,34,0,1,2,0,0,0,37,0,Abruzzo
1,2020-12-27,106,41,65,104,0,2,0,0,0,0,106,0,Basilicata
2,2020-12-27,275,150,125,274,0,1,0,0,0,0,275,0,Calabria
3,2020-12-27,720,422,298,703,14,3,0,0,0,0,720,0,Campania
4,2020-12-27,968,372,596,824,1,117,26,0,0,0,968,0,Emilia-Romagna
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773,2021-03-24,7572,3624,3948,350,5,3979,232,2618,35,353,4920,2652,Sicilia
1774,2021-03-24,4599,1869,2730,51,157,818,39,3401,34,99,2099,2500,Toscana
1775,2021-03-24,1824,889,935,108,76,90,4,1493,31,22,1021,803,Umbria
1776,2021-03-24,212,96,116,2,0,38,0,169,2,1,11,201,Valle d'Aosta / Vallée d'Aoste


In [15]:
somVacciniSumLat.groupby(['Regione','Data Somministrazione']).sum()

Totale  Sesso Maschile  Sesso Femminile  \
Regione Data Somministrazione                                            
Abruzzo 2020-12-27                 37              19               18   
        2021-01-02                548             218              330   
        2021-01-03                471             185              286   
        2021-01-04               1401             575              826   
        2021-01-05               1614             604             1010   
...                               ...             ...              ...   
Veneto  2021-03-20              12309            5068             7241   
        2021-03-21              11904            4683             7221   
        2021-03-22              17709            7865             9844   
        2021-03-23              20878            9419            11459   
        2021-03-24               8364            3720             4644   

                               Categoria Operatori Sanitari Sociosanitari  \
Regione Data Somministrazione                                               
Abruzzo 2020-12-27                                                     34   
        2021-01-02                                                    540   
        2021-01-03                                                    464   
        2021-01-04                                                   1396   
        2021-01-05                                                   1604   
...                                                                   ...   
Veneto  2021-03-20                                                    636   
        2021-03-21                                                   1470   
        2021-03-22                                                   1136   
        2021-03-23                                                    962   
        2021-03-24                                                    541   

                               Categoria Personale Non Sanitario  \
Regione Data Somministrazione                                      
Abruzzo 2020-12-27                                             0   
        2021-01-02                                             0   
        2021-01-03                                             0   
        2021-01-04                                             0   
        2021-01-05                                             0   
...                                                          ...   
Veneto  2021-03-20                                            25   
        2021-03-21                                            50   
        2021-03-22                                           115   
        2021-03-23                                           171   
        2021-03-24                                           161   

                               Categoria Altro  Categoria Ospiti Rsa  \
Regione Data Somministrazione                                          
Abruzzo 2020-12-27                           1                     2   
        2021-01-02                           8                     0   
        2021-01-03                           7                     0   
        2021-01-04                           5                     0   
        2021-01-05                          10                     0   
...                                        ...                   ...   
Veneto  2021-03-20                        4459                   199   
        2021-03-21                        3144                    26   
        2021-03-22                        6461                   452   
        2021-03-23                        9308                   530   
        2021-03-24                        1397                   467   

                               Categoria Over80  Categoria Forze Armate  \
Regione Data Somministrazione                                             
Abruzzo 2020-12-27                            0                       0   
        2021-01-02 

In [26]:
abitantiregioni = pd.read_csv('script/DCIS_POPRES1_25022021122609782.csv')
abitantiregioni = abitantiregioni.iloc[:,[1,5,6,9,12]].sort_values(['Territorio',
                                                    'Sesso']
                                                  ).where((abitantiregioni['Stato civile']=='totale') &
                                                          (abitantiregioni.Sesso=='totale') &
                                                          (abitantiregioni.ETA1=='TOTAL')
                                                         ).dropna().iloc[:,[0,-1]].set_index('Territorio')
# Rename field and indexfor better naming
abitantiregioni = abitantiregioni.rename(columns={'Value': 'Abitanti'}).rename_axis('Regione').astype(int)
abitantiregioni

,Abitanti
Regione,
Abruzzo,1293941
Basilicata,553254
Calabria,1894110
Campania,5712143
Centro,11831092
Emilia-Romagna,4464119
Friuli-Venezia Giulia,1206216
Isole,6486911
Italia,59641488


In [29]:
anaVacSumLat

,Fascia Anagrafica,Totale,Sesso Maschile,Sesso Femminile,Categoria Operatori Sanitari Sociosanitari,Categoria Personale Non Sanitario,Categoria Altro,Categoria Ospiti Rsa,Categoria Over80,Categoria Forze Armate,Categoria Personale Scolastico,Prima Dose,Seconda Dose,% Seconda Dose Sul Totale,Platea,% Seconda Dose Assoluta,% Totale Assoluto
0,16-19,8878,4250,4628,4006,658,2896,1159,0,54,105,6039,2839,31.98,2298846,0.12,0.39
1,20-29,591954,210451,381503,405492,40346,65085,9600,0,17821,53610,368260,223694,37.79,6084382,3.68,9.73
2,30-39,857977,335737,522240,520665,67754,104846,12265,0,33415,119032,557215,300762,35.05,6854632,4.39,12.52
3,40-49,1152749,403571,749178,602555,106262,153670,18390,0,61895,209977,783284,369465,32.05,8937229,4.13,12.90
4,50-59,1486370,528941,957429,747095,152362,198238,30500,0,83247,274928,1004397,481973,32.43,9414195,5.12,15.79
5,60-69,910437,430285,480152,500964,75216,145997,37416,0,11553,139291,598904,311533,34.22,7364364,4.23,12.36
6,70-79,387671,220877,166794,99295,15134,202561,68433,0,99,2149,294041,93630,24.15,5968373,1.57,6.50
7,80-89,2332287,997986,1334301,13948,6444,64246,186667,2060443,326,213,1658878,673409,28.87,3628160,18.56,64.28
8,90+,618122,188611,429511,2758,2151,12836,158467,441805,71,34,417840,200282,32.40,613523,32.64,100.75
